In [1]:
def check(res, test):
    k = 0
    success = 0
    for a in test.id:
        
        #print a, names[res[k]], test[test.id == a].author.values
        if names[res[k]] == test[test.id == a].author.values:
            success+=1
        k+=1
    print success, len(test), float(success) / float(len(test))
    return float(success) / float(len(test))

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
tr_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
names = {0 : 'EAP', 1 : 'HPL', 2 : 'MWS'}

quant = int(0.7 * float(len(tr_data)))
train = tr_data[:quant]
test = tr_data[quant:]
#print train.head(10)
#print len(train)



In [153]:
#naive Bayes

vectorizer = CountVectorizer(min_df=0 , token_pattern=r'[^a-zA-Z\']')
texts_cv = vectorizer.fit_transform(tr_data.text)
texts_train_cv = vectorizer.transform (train.text)
texts_test_cv = vectorizer.transform(test.text)
#print texts_test_cv

from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(texts_train_cv, train.author)
clf.score(texts_test_cv, test.author)

texts_test = vectorizer.transform(test.text)
clf=MultinomialNB()
clf.fit(texts_cv, tr_data.author)
predicted_result=clf.predict_proba(texts_test)
res = predicted_result.argmax(axis = 1)

check(res, test)


2964 5874 0.504596527068


0.5045965270684372

In [4]:
#logistic regression

import matplotlib.pyplot as plt
from sklearn import linear_model, decomposition

slvrs = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
X = texts_cv
Y = tr_data.author

#h = .02  # step size in the mesh
for s in slvrs:
    logreg = linear_model.LogisticRegression(C=1e20, solver = s)
    auth = []
    for i in train.author:
        a = [b for b in range(3) if names[b] == i]
        auth.append(a)
    logreg.fit(texts_train_cv, auth)

    Z = logreg.predict(texts_test)
    check(Z, test)

C:\Users\1\Anaconda2\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


3019 5874 0.513959822949
3004 5874 0.511406196799
3018 5874 0.513789581205


C:\Users\1\Anaconda2\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


2768 5874 0.471229145386
2639 5874 0.449267960504


In [7]:
#Stoch Grad

from sklearn.linear_model import SGDClassifier
auth = []
for i in train.author:
    a = [b for b in range(3) if names[b] == i]
    auth.append(a)
l = ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']
for _l in l:
    for _alpha in range(5):
        for eps in range(1, 5, 1):
          #  print eps
            clf = SGDClassifier(loss=_l, penalty="l2", alpha = 10 ** (-_alpha), epsilon = eps * 0.1)
            clf.fit(texts_train_cv, auth)
            a = clf.predict(texts_test)
            print _l, _alpha, eps
            check(a, test)
            
#epsilon_insensitive alpha = 0.1 eps = 0.2: 0.506469186244

C:\Users\1\Anaconda2\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


hinge 0 1
2720 5874 0.463057541709
hinge 0 2
2506 5874 0.426625808648
hinge 0 3
2567 5874 0.437010554988
hinge 0 4
2429 5874 0.413517194416
hinge 1 1
2728 5874 0.464419475655
hinge 1 2
2562 5874 0.436159346272
hinge 1 3
2909 5874 0.495233231188
hinge 1 4
2828 5874 0.481443649983
hinge 2 1
2259 5874 0.384576098059
hinge 2 2
2541 5874 0.432584269663
hinge 2 3
2707 5874 0.460844399047
hinge 2 4
2417 5874 0.411474293497
hinge 3 1
2505 5874 0.426455566905
hinge 3 2
2084 5874 0.354783792986
hinge 3 3
2813 5874 0.478890023834
hinge 3 4
2540 5874 0.43241402792
hinge 4 1
2125 5874 0.36176370446
hinge 4 2
2129 5874 0.362444671433
hinge 4 3
1888 5874 0.321416411304
hinge 4 4
2639 5874 0.449267960504
log 0 1
2531 5874 0.43088185223
log 0 2
2441 5874 0.415560095335
log 0 3
2406 5874 0.409601634321
log 0 4
2385 5874 0.406026557712
log 1 1
2785 5874 0.474123255022
log 1 2
2493 5874 0.424412665986
log 1 3
2433 5874 0.414198161389
log 1 4
2661 5874 0.453013278856
log 2 1
2383 5874 0.405686074225
log 2 

In [154]:
def print_top_words(model, feature_names, n_top_words):
    for index, topic in enumerate(model.components_):
        message = "\nTopic #{}:".format(index)
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1 :-1]])
        print(message)
        print("="*70)
      
import nltk
lemm = nltk.WordNetLemmatizer()
class LemmaCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(LemmaCountVectorizer, self).build_analyzer()
        return lambda doc: (lemm.lemmatize(w) for w in analyzer(doc))
  
# Storing the entire training text in a list
text = list(train.text.values)
# Calling our overwritten Count vectorizer
nltk.download('wordnet')
tf_vectorizer = LemmaCountVectorizer(max_df=0.95, 
                                     min_df=2,
                                     stop_words='english',
                                     decode_error='ignore')
tf = tf_vectorizer.fit_transform(text)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [155]:
#LDA
tf_authors = {0 : [], 1 : [], 2 : []}
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=3, max_iter=5,
                                learning_method = 'online',
                                learning_offset = 50.,
                                random_state = 0)
lda.fit(tf)
n_top_words = 40
print("\nTopics in LDA model: ")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

train_topics = np.matrix(lda.transform(tf))/np.matrix(lda.transform(tf)).sum(axis = 1)




Topics in LDA model: 

Topic #0:thing said great say soon did man matter mr little earth mind friend year tell left thousand know make right having age sun arm general old character book certain given possible poor appearance world case letter course sure good fully

Topic #1:night house like old came heard saw room time door hour light strange dream street sea year sound wall length water window open place looked long half man day away seen little city head small body left dark lay tree

Topic #2:life day time thought eye heart word man shall did love death said let father long hope raymond work come child feeling soul friend voice felt mind think know hand moment spirit world idea horror fear nature way lost scene


In [92]:
topics_match = np.zeros((3,3))
#print train_topics[0].max()
k = 0
#print [j for j in range(3) if names[j] == 'HPL'][0]
for a in train.author:
    au = [j for j in range(3) if names[j] == a][0] #true author num
    top = train_topics[k].argmax() #topic num
    #print au, top
    topics_match[au][top] +=1
    k+=1
print topics_match
#Here we can see 0th topic is 0th author etc.



[[2177. 1670. 1673.]
 [ 751. 2471.  725.]
 [ 637.  911. 2690.]]


In [93]:
tf_test = tf_vectorizer.transform(list(test.text.values))
doc_topic_dist_unnormalized = np.matrix(lda.transform(tf_test))
doc_topic_dist = doc_topic_dist_unnormalized/doc_topic_dist_unnormalized.sum(axis=1)
res = []
for i in doc_topic_dist:
    res.append(i.argmax())
check(res, test)

3250 5874 0.553285665645


0.5532856656452162

In [107]:
for n in range(4, 20):
 
    lda = LatentDirichletAllocation(n_components=n, max_iter=5,
                                    learning_method = 'online',
                                    learning_offset = 50.,
                                    random_state = 0)
    lda.fit(tf)
    n_top_words = 40
  #  print("\nTopics in LDA model: ")
    tf_feature_names = tf_vectorizer.get_feature_names()
   # print_top_words(lda, tf_feature_names, n_top_words)

    train_topics = np.matrix(lda.transform(tf))/np.matrix(lda.transform(tf)).sum(axis = 1)

    topics_match = np.zeros((3,n))
    #print train_topics[0].max()
    k = 0
    #print [j for j in range(3) if names[j] == 'HPL'][0]
    for a in train.author:
        au = [j for j in range(3) if names[j] == a][0] #true author num
        top = train_topics[k].argmax() #topic num
        #print au, top
        topics_match[au][top] +=1
        k+=1
   # print topics_match
   # print topics_match.argmax(axis = 0)
    maxes = topics_match.argmax(axis = 0)

    tf_test = tf_vectorizer.transform(list(test.text.values))
    doc_topic_dist_unnormalized = np.matrix(lda.transform(tf_test))
    doc_topic_dist = doc_topic_dist_unnormalized/doc_topic_dist_unnormalized.sum(axis=1)
    res = []
    for i in doc_topic_dist:
        res.append(maxes[i.argmax()])
    #print res
    print n, ':'
    check(res, test)



4 :
2662 5874 0.453183520599
5 :
2658 5874 0.452502553626
6 :
2988 5874 0.508682328907
7 :
2732 5874 0.465100442629
8 :
2696 5874 0.458971739871
9 :
2895 5874 0.492849846782
10 :
2671 5874 0.454715696289
11 :
2488 5874 0.423561457269
12 :
2590 5874 0.440926115083
13 :
2585 5874 0.440074906367
14 :
2646 5874 0.450459652707
15 :
2733 5874 0.465270684372
16 :
2739 5874 0.466292134831
17 :
2634 5874 0.448416751788
18 :
2575 5874 0.438372488934
19 :
2700 5874 0.459652706844


In [109]:
for n in range(5, 20):
 
    lda = LatentDirichletAllocation(n_components=3, max_iter=n,
                                    learning_method = 'online',
                                    learning_offset = 50.,
                                    random_state = 0)
    lda.fit(tf)
    n_top_words = 40
  #  print("\nTopics in LDA model: ")
    tf_feature_names = tf_vectorizer.get_feature_names()
   # print_top_words(lda, tf_feature_names, n_top_words)

    train_topics = np.matrix(lda.transform(tf))/np.matrix(lda.transform(tf)).sum(axis = 1)

    topics_match = np.zeros((3,3))
    #print train_topics[0].max()
    k = 0
    #print [j for j in range(3) if names[j] == 'HPL'][0]
    for a in train.author:
        au = [j for j in range(3) if names[j] == a][0] #true author num
        top = train_topics[k].argmax() #topic num
        #print au, top
        topics_match[au][top] +=1
        k+=1
   # print topics_match
   # print topics_match.argmax(axis = 0)
    maxes = topics_match.argmax(axis = 0)

    tf_test = tf_vectorizer.transform(list(test.text.values))
    doc_topic_dist_unnormalized = np.matrix(lda.transform(tf_test))
    doc_topic_dist = doc_topic_dist_unnormalized/doc_topic_dist_unnormalized.sum(axis=1)
    res = []
    for i in doc_topic_dist:
        res.append(maxes[i.argmax()])
    #print res
    print n, ':'
    check(res, test)



5 :
3250 5874 0.553285665645
6 :
3284 5874 0.559073884917
7 :
3326 5874 0.566224038134
8 :
3358 5874 0.571671773919
9 :
3374 5874 0.574395641811
10 :
3397 5874 0.578311201907
11 :
3412 5874 0.580864828056
12 :
3432 5874 0.584269662921
13 :
3437 5874 0.585120871638
14 :
3443 5874 0.586142322097
15 :
3458 5874 0.588695948247
16 :
3462 5874 0.58937691522
17 :
3469 5874 0.590568607423
18 :
3472 5874 0.591079332652
19 :
3486 5874 0.593462717058


In [115]:
#NMF
from sklearn.decomposition import NMF
solver = ['cd', 'mu']
bloss = ['frobenius', 'kullback-leibler', 'itakura-saito']
for s in solver:
    for b in bloss:
        if s == 'cd' and b != 'frobenius':
            continue
        model = NMF(n_components=3, solver = s, beta_loss = b, init='random', random_state=0)
        W = model.fit(tf)
        train_topics = np.matrix(lda.transform(tf))/np.matrix(lda.transform(tf)).sum(axis = 1)

        topics_match = np.zeros((3,3))
            #print train_topics[0].max()
        k = 0
        #print [j for j in range(3) if names[j] == 'HPL'][0]
        for a in train.author:
            au = [j for j in range(3) if names[j] == a][0] #true author num
            top = train_topics[k].argmax() #topic num
            #print au, top
            topics_match[au][top] +=1
            k+=1
           # print topics_match
           # print topics_match.argmax(axis = 0)
        maxes = topics_match.argmax(axis = 0)

        tf_test = tf_vectorizer.transform(list(test.text.values))
        doc_topic_dist_unnormalized = np.matrix(lda.transform(tf_test))
        doc_topic_dist = doc_topic_dist_unnormalized/doc_topic_dist_unnormalized.sum(axis=1)
        res = []
        for i in doc_topic_dist:
            res.append(maxes[i.argmax()])
        #print res
        print s, b, ':'
        check(res, test)

cd frobenius :
3486 5874 0.593462717058
mu frobenius :
3486 5874 0.593462717058
mu kullback-leibler :
3486 5874 0.593462717058


C:\Users\1\Anaconda2\lib\site-packages\sklearn\decomposition\nmf.py:156: RuntimeWarning:

invalid value encountered in sqrt



mu itakura-saito :
3486 5874 0.593462717058


In [134]:
#Random forest
from sklearn.ensemble import RandomForestClassifier
for es in range(10, 60, 10):
    for d in range(5, 9):
        rfc = RandomForestClassifier(n_estimators=es, max_depth=d, random_state=42, n_jobs=-1)
        auth = []
        for i in train.author:
            a = [b for b in range(3) if names[b] == i]
            auth.append(a)
            # we create an instance of Neighbours Classifier and fit the data.

        rfc.fit(texts_train_cv, auth)
        pred_rfc=rfc.predict(texts_test)
        print es, d
        check(pred_rfc, test)

C:\Users\1\Anaconda2\lib\site-packages\ipykernel_launcher.py:12: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



10 5
2989 5874 0.50885257065
10 6
3018 5874 0.513789581205
10 7
3044 5874 0.51821586653
10 8
2990 5874 0.509022812394
20 5
2994 5874 0.509703779367
20 6
3010 5874 0.512427647259
20 7
3037 5874 0.517024174328
20 8
3040 5874 0.517534899557
30 5
3045 5874 0.518386108274
30 6
3009 5874 0.512257405516
30 7
3057 5874 0.520429009193
30 8
3037 5874 0.517024174328
40 5
2990 5874 0.509022812394
40 6
3005 5874 0.511576438543
40 7
3045 5874 0.518386108274
40 8
3044 5874 0.51821586653
50 5
3006 5874 0.511746680286
50 6
3058 5874 0.520599250936
50 7
3056 5874 0.52025876745
50 8
3046 5874 0.518556350017


In [137]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators':[10, 20, 30, 40, 50],  'max_depth':[4,5,6,7,8]}

grid_search_rfc = GridSearchCV(RandomForestClassifier(random_state=42,n_jobs=-1), param_grid, cv=7)

tr_auth = []
for i in train.author:
    a = [b for b in range(3) if names[b] == i]
    tr_auth.append(a)
grid_search_rfc.fit(texts_train_cv, tr_auth)

t_auth = []
for i in test.author:
    a = [b for b in range(3) if names[b] == i]
    t_auth.append(a)

print(grid_search_rfc.best_params_,grid_search_rfc.best_score_)
rfc = RandomForestClassifier(n_estimators=grid_search_rfc.best_params_['n_estimators'], max_depth=grid_search_rfc.best_params_['max_depth'], random_state=42, n_jobs=-1)
auth = []
for i in train.author:
    a = [b for b in range(3) if names[b] == i]
    auth.append(a)
            # we create an instance of Neighbours Classifier and fit the data.

rfc.fit(texts_train_cv, auth)
pred_rfc=rfc.predict(texts_test)
check(pred_rfc, test)


({'n_estimators': 30, 'max_depth': 8}, 0.5284932506384531)


C:\Users\1\Anaconda2\lib\site-packages\ipykernel_launcher.py:25: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



50 8
3037 5874 0.517024174328


0.5170241743275451

In [161]:
#TF-IDF model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(train.text) + list(test.text))
xtrain_tfv =  tfv.transform(train.text) 
xvalid_tfv = tfv.transform(test.text)

In [162]:
#logreg
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, tr_auth)
predictions = clf.predict_proba(xvalid_tfv)
res = predictions.argmax(axis = 1)
check(res, test)

4666 5874 0.794347974123


0.794347974123255

In [150]:
#naive bayes
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()
mnb.fit(xtrain_tfv, tr_auth)

predicted_result = mnb.predict_proba(xvalid_tfv)
res = predicted_result.argmax(axis = 1)
check(res, test)

4819 5874 0.820394960844


0.820394960844399

In [158]:
#Stoch grad
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {'loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
              'alpha':[1e-1, 1e-2, 1e-3, 1e-4],
              'epsilon':[0.1, 0.2, 0.3, 0.4, 0.5]             
             }

         
grid_search_rfc = GridSearchCV(SGDClassifier(penalty="l2"), param_grid, cv=7)

grid_search_rfc.fit(xtrain_tfv, tr_auth)

print(grid_search_rfc.best_params_,grid_search_rfc.best_score_)

sgd = SGDClassifier(penalty="l2", alpha = grid_search_rfc.best_params_['alpha'], loss = grid_search_rfc.best_params_['loss'], epsilon = grid_search_rfc.best_params_['epsilon'])
clf.fit(xtrain_tfv, tr_auth)
a = clf.predict(xvalid_tfv)
check(a, test)

({'alpha': 0.0001, 'loss': 'hinge', 'epsilon': 0.1}, 0.8)
4819 5874 0.820394960844


0.820394960844399

In [160]:
#rendom forest
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators':[10, 20, 30, 40, 50],  'max_depth':[4,5,6,7,8]}

grid_search_rfc = GridSearchCV(RandomForestClassifier(random_state=42,n_jobs=-1), param_grid, cv=7)

tr_auth = []
for i in train.author:
    a = [b for b in range(3) if names[b] == i]
    tr_auth.append(a)
grid_search_rfc.fit(xtrain_tfv, tr_auth)

t_auth = []
for i in test.author:
    a = [b for b in range(3) if names[b] == i]
    t_auth.append(a)

print(grid_search_rfc.best_params_,grid_search_rfc.best_score_)
rfc = RandomForestClassifier(n_estimators=grid_search_rfc.best_params_['n_estimators'], max_depth=grid_search_rfc.best_params_['max_depth'], random_state=42, n_jobs=-1)
auth = []
for i in train.author:
    a = [b for b in range(3) if names[b] == i]
    auth.append(a)
            # we create an instance of Neighbours Classifier and fit the data.

rfc.fit(xtrain_tfv, tr_auth)
pred_rfc=rfc.predict(xvalid_tfv)
check(pred_rfc, test)


({'n_estimators': 10, 'max_depth': 8}, 0.45457862094126233)


C:\Users\1\Anaconda2\lib\site-packages\ipykernel_launcher.py:25: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



2800 5874 0.476676881171


0.47667688117126317

In [164]:
lda = LatentDirichletAllocation(n_components=3, max_iter=5,
                                    learning_method = 'online',
                                    learning_offset = 50.,
                                    random_state = 0)
lda.fit(xtrain_tfv)
n_top_words = 40
  #  print("\nTopics in LDA model: ")
tf_feature_names = tfv.get_feature_names()
   # print_top_words(lda, tf_feature_names, n_top_words)

train_topics = np.matrix(lda.transform(xtrain_tfv))/np.matrix(lda.transform(xtrain_tfv)).sum(axis = 1)

topics_match = np.zeros((3,3))
    #print train_topics[0].max()
k = 0
    #print [j for j in range(3) if names[j] == 'HPL'][0]
for a in train.author:
    au = [j for j in range(3) if names[j] == a][0] #true author num
    top = train_topics[k].argmax() #topic num
        #print au, top
    topics_match[au][top] +=1
    k+=1
   # print topics_match
   # print topics_match.argmax(axis = 0)
maxes = topics_match.argmax(axis = 0)

doc_topic_dist_unnormalized = np.matrix(lda.transform(xvalid_tfv))
doc_topic_dist = doc_topic_dist_unnormalized/doc_topic_dist_unnormalized.sum(axis=1)
res = []
for i in doc_topic_dist:
    res.append(maxes[i.argmax()])
    #print res

check(res, test)

2416 5874 0.411304051753


0.41130405175348994

In [165]:
from sklearn.decomposition import NMF
solver = ['cd', 'mu']
bloss = ['frobenius', 'kullback-leibler', 'itakura-saito']
for s in solver:
    for b in bloss:
        if s == 'cd' and b != 'frobenius':
            continue
        model = NMF(n_components=3, solver = s, beta_loss = b, init='random', random_state=0)
        W = model.fit(xtrain_tfv)
        train_topics = np.matrix(lda.transform(xtrain_tfv))/np.matrix(lda.transform(xtrain_tfv)).sum(axis = 1)

        topics_match = np.zeros((3,3))
            #print train_topics[0].max()
        k = 0
        #print [j for j in range(3) if names[j] == 'HPL'][0]
        for a in train.author:
            au = [j for j in range(3) if names[j] == a][0] #true author num
            top = train_topics[k].argmax() #topic num
            #print au, top
            topics_match[au][top] +=1
            k+=1
           # print topics_match
           # print topics_match.argmax(axis = 0)
        maxes = topics_match.argmax(axis = 0)

        tf_test = tfv.transform(list(test.text.values))
        doc_topic_dist_unnormalized = np.matrix(lda.transform(tf_test))
        doc_topic_dist = doc_topic_dist_unnormalized/doc_topic_dist_unnormalized.sum(axis=1)
        res = []
        for i in doc_topic_dist:
            res.append(maxes[i.argmax()])
        #print res
        print s, b, ':'
        check(res, test)

cd frobenius :
2416 5874 0.411304051753
mu frobenius :
2416 5874 0.411304051753
mu kullback-leibler :
2416 5874 0.411304051753
mu itakura-saito :
2416 5874 0.411304051753
